In [1]:
from scipy import signal
from scipy.fftpack import fft
import uconnrcmpy as ucr
import numpy as np
from pathlib import Path
from subprocess import run
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset, BboxConnector

# %matplotlib

In [2]:
plt.rc('font', family='serif', serif='Times New Roman', size=22.0)
dpi = '300'

# Draw the Raw Voltage Trace figure

In [ ]:
volt_trace = ucr.traces.VoltageTrace(Path('10_in_00_mm_333K-979t-100x-13-Jul-15-1349.txt'))
raw_fig, raw_ax = plt.subplots(figsize=[14, 9])
raw_ax.plot(volt_trace.time*1000, volt_trace.signal[:, 1], label="Original")
raw_ax.plot(volt_trace.time*1000, volt_trace.filtered_voltage, label="Filtered")
raw_ax.plot(volt_trace.time*1000, volt_trace.smoothed_voltage, label="Filtered & Smoothed")
raw_ax.set_xlim([160, 240])
raw_ax.set_ylim([-0.01, 1.01])
raw_ax.set_xlabel('Time [ms]')
raw_ax.set_ylabel('Voltage [V]')
raw_ax_legend = raw_ax.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                              ncol=3, mode="expand", borderaxespad=0.)

for leg in raw_ax_legend.legendHandles:
    leg.set_linewidth(2.0)

# Create the zoomed inset
raw_inset = zoomed_inset_axes(raw_ax, 18, loc=6, bbox_to_anchor=raw_ax.transData.transform((165, 0.6)))
raw_inset.plot(volt_trace.time*1000, volt_trace.signal[:, 1])
raw_inset.plot(volt_trace.time*1000, volt_trace.filtered_voltage, linewidth=3.0)
raw_inset.plot(volt_trace.time*1000, volt_trace.smoothed_voltage)
raw_inset.set_xlim([190.5, 194])
raw_inset.set_ylim([0.27, 0.29])
xticks = raw_inset.get_xticks()
raw_inset.set_xticks(xticks[1:-1])

pp, p1, p2 = mark_inset(raw_ax, raw_inset, loc1=3, loc2=4, fc="none", ec="0.5")

fig_name = 'raw-voltage'

raw_fig.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

# This gets drawn in front of the inset axes, when it should be behind for the proper effect :(
# p3 = BboxConnector(raw_inset.bbox, pp.bbox, loc1=2, loc2=2, fc="none", ec="0.5")
# raw_inset.add_patch(p3)
# p3.set_clip_on(False)

# Plot the 2-stage ignition case

In [ ]:
expt = ucr.dataprocessing.Experiment(Path('00_in_02_mm_373K-1282t-100x-19-Jul-15-1633.txt'))
nr_expt = ucr.dataprocessing.Experiment(Path('NR_00_in_02_mm_373K-1278t-100x-19-Jul-15-1652.txt'))

In [ ]:
ign_fig, pressure_ax = plt.subplots(figsize=[14, 9])
pressure_ax.plot(expt.pressure_trace.zeroed_time*1000,
                 expt.pressure_trace.pressure,
                 linewidth=1.5, label="Pressure")
pressure_ax.plot(nr_expt.pressure_trace.zeroed_time*1000,
                 nr_expt.pressure_trace.pressure,
                 linewidth=1.5, label="Non-Reactive Pressure")
dpdt_ax = pressure_ax.twinx()
dpdt_ax.plot(expt.pressure_trace.zeroed_time*1000,
             expt.pressure_trace.smoothed_derivative/1000,
             'r', linewidth=1.5, label="Time Derivative of Pressure")

dpdt_ax.set_ylim(-0.25, 3)
dpdt_ax.set_ylabel('Time Derivative of Pressure [bar/ms]')

pressure_ax.axvline(x=0, color='k', ymax=0.52, linewidth=1.5)
pressure_ax.axvline(x=expt.ignition_delay, color='k', linewidth=1.5)
pressure_ax.axvline(x=expt.first_stage, color='k', ymax=0.25, linewidth=1.5)
pressure_ax.annotate('', xy=(0, 10), xycoords='data', xytext=(expt.first_stage, 10), textcoords='data',
                     arrowprops={'arrowstyle': '<|-|>', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black', 'linewidth': 1.5})
pressure_ax.annotate('$τ_1$', xy=(expt.first_stage/2, 10), xycoords='data', xytext=(0, 10), textcoords='offset points')
pressure_ax.annotate('$τ$', xy=(expt.ignition_delay/2, 20), xycoords='data', xytext=(0, 10), textcoords='offset points')
pressure_ax.annotate('', xy=(0, 20), xycoords='data', xytext=(expt.ignition_delay, 20), textcoords='data',
                     arrowprops={'arrowstyle': '<|-|>', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black', 'linewidth': 1.5})
pressure_ax.annotate('EOC', xy=(0, 31.5), xycoords='data')
pressure_ax.annotate('', xy=(-4, 15), xycoords='data', xytext=(-17, 15), textcoords='data',
                     arrowprops={'arrowstyle': '<|-', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black', 'linewidth': 1.5})
dpdt_ax.annotate('', xy=(-1, 1.75), xycoords='data', xytext=(12, 1.75), textcoords='data',
                 arrowprops={'arrowstyle': '<|-', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black', 'linewidth': 1.5})
pressure_ax.set_ylim(0, 60)
pressure_ax.set_xlim(-30, 70)
pressure_ax.set_xlabel('Time [ms]')
pressure_ax.set_ylabel('Pressure [bar]')

lin_1, leg_1 = pressure_ax.get_legend_handles_labels()
lin_2, leg_2 = dpdt_ax.get_legend_handles_labels()
dpdt_ax_legend = dpdt_ax.legend(lin_1 + lin_2, leg_1 + leg_2, bbox_to_anchor=(0.85, 0.95))

for leg in dpdt_ax_legend.legendHandles:
    leg.set_linewidth(2.0)

fig_name = 'ign-delay-def'

ign_fig.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])
# pressure_inset = zoomed_inset_axes(pressure_ax, 4, loc=9)
# pressure_inset.plot(expt.pressure_trace.zeroed_time*1000, expt.pressure_trace.pressure)
# pressure_inset.set_xlim(30, 40)
# pressure_inset.set_ylim(27, 30)

# dpdt_inset = pressure_inset.twinx()
# dpdt_inset.plot(expt.pressure_trace.zeroed_time*1000, expt.pressure_trace.smoothed_derivative/1000, 'g')
# # dpdt_inset.set_ylim(0, 5)


# Draw the frequency spectrum

In [ ]:
volt_trace = ucr.traces.VoltageTrace(Path('10_in_00_mm_333K-979t-100x-13-Jul-15-1349.txt'))
samp_freq = np.ceil(1/volt_trace.time[1])

freq_fig, freq_ax = plt.subplots(figsize=[14, 9])    
f, orig_den = signal.welch(volt_trace.signal[:, 1], fs=samp_freq, nperseg=2**14)
freq_ax.plot(f, orig_den, label='Original')
f, filt_den = signal.welch(volt_trace.filtered_voltage, fs=samp_freq, nperseg=2**14)
freq_ax.plot(f, filt_den, label='Filtered')
f, sm_den = signal.welch(volt_trace.smoothed_voltage, fs=samp_freq, nperseg=2**14)
freq_ax.plot(f, sm_den, label='Filtered & Smoothed')

freq_ax.set_ylim(1.0E-19, 1.0E-4)
freq_ax.set_ylabel('Power Spectral Density [W/Hz]')
freq_ax.set_xlabel('Frequency [Hz]')
freq_ax.set_yscale('log')
freq_ax_legend = freq_ax.legend(loc='best')

for leg in freq_ax_legend.legendHandles:
    leg.set_linewidth(2.0)

fig_name = 'frequency'

freq_fig.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

# Run the Example

In [3]:
# %matplotlib inline
x_lo = -0.04
x_hi = 0.075
cond_00_in_02_mm = ucr.Condition()
cond_00_in_02_mm.add_experiment(Path('00_in_02_mm_373K-1285t-100x-19-Jul-15-1620.txt'))
cond_00_in_02_mm.add_experiment(Path('00_in_02_mm_373K-1282t-100x-19-Jul-15-1626.txt'))
cond_00_in_02_mm.add_experiment(Path('00_in_02_mm_373K-1282t-100x-19-Jul-15-1633.txt'))
cond_00_in_02_mm.add_experiment(Path('00_in_02_mm_373K-1282t-100x-19-Jul-15-1640.txt'))
cond_00_in_02_mm.add_experiment(Path('00_in_02_mm_373K-1282t-100x-19-Jul-15-1646.txt'))
cond_00_in_02_mm.add_experiment(Path('NR_00_in_02_mm_373K-1278t-100x-19-Jul-15-1652.txt'))
cond_00_in_02_mm.create_volume_trace()
cond_00_in_02_mm.compare_to_sim(run_reactive=True, run_nonreactive=True)

759	46.116253


In [ ]:
fig_name = 'all-runs'
cond_00_in_02_mm.all_runs_figure.set_size_inches(14, 9)
cond_00_in_02_mm.all_runs_axis.set_xlim(x_lo, x_hi)
cond_00_in_02_mm.all_runs_figure.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

In [ ]:
fig_name = 'one-run'
exp_fig = cond_00_in_02_mm.reactive_experiments['00_in_02_mm_373K-1282t-100x-19-Jul-15-1633.txt'].exp_fig
exp_fig.set_size_inches(14, 9)
cond_00_in_02_mm.reactive_experiments['00_in_02_mm_373K-1282t-100x-19-Jul-15-1633.txt'].p_axis.set_xlim(x_lo, x_hi)
exp_fig.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

In [ ]:
fig_name = 'nonreactive-run'
cond_00_in_02_mm.nonreactive_figure.set_size_inches(14, 9)
cond_00_in_02_mm.nonreactive_axis.set_xlim(x_lo, x_hi)
cond_00_in_02_mm.nonreactive_figure.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

In [ ]:
fig_name = 'pressure-comparison'
cond_00_in_02_mm.pressure_comparison_figure.set_size_inches(14, 9)
cond_00_in_02_mm.pressure_comparison_axis.set_xlim(-0.036, -0.023)
cond_00_in_02_mm.pressure_comparison_axis.set_ylim(1.68, 1.85)
cond_00_in_02_mm.pressure_comparison_figure.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

In [5]:
fig_name = 'simulation-comparison'
cond_00_in_02_mm.simulation_figure.set_size_inches(14, 9)
cond_00_in_02_mm.simulation_axis.set_xlim(x_lo*1000, x_hi*1000)
cond_00_in_02_mm.simulation_axis.legend(loc='best')
cond_00_in_02_mm.simulation_figure.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

CompletedProcess(args=['convert', '-density', '300', 'simulation-comparison.pdf', 'simulation-comparison.png'], returncode=0)